In [13]:
!pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 15.3 MB/s eta 0:00:00


In [14]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import gensim
from sklearn.linear_model import LogisticRegression
import pickle
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import spacy
import string
import re
import emoji


In [15]:
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [16]:
# Load Bitcoin Tweet Dataset
tweets_df = pd.read_csv('/content/drive/MyDrive/NLP/Bitcoin_tweets.csv', nrows=10000)

# Load Bitcoin Price Dataset
price_df = pd.read_csv('/content/drive/MyDrive/NLP/bitcoin_2017_to_2023.csv')



In [17]:
tweets_df.head()


,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,DeSota Wilson,"Atlanta, GA","Biz Consultant, real estate, fintech, startups...",2009-04-26 20:05:09,8534.0,7605,4838,False,2021-02-10 23:59:04,Blue Ridge Bank shares halted by NYSE after #b...,['bitcoin'],Twitter Web App,False
1,CryptoND,NaN,😎 BITCOINLIVE is a Dutch platform aimed at inf...,2019-10-17 20:12:10,6769.0,1532,25483,False,2021-02-10 23:58:48,"😎 Today, that's this #Thursday, we will do a ""...","['Thursday', 'Btc', 'wallet', 'security']",Twitter for Android,False
2,Tdlmatias,"London, England","IM Academy : The best #forex, #SelfEducation, ...",2014-11-10 10:50:37,128.0,332,924,False,2021-02-10 23:54:48,"Guys evening, I have read this article about B...",NaN,Twitter Web App,False
3,Crypto is the future,NaN,I will post a lot of buying signals for BTC tr...,2019-09-28 16:48:12,625.0,129,14,False,2021-02-10 23:54:33,$BTC A big chance in a billion! Price: \487264...,"['Bitcoin', 'FX', 'BTC', 'crypto']",dlvr.it,False
4,Alex Kirchmaier 🇦🇹🇸🇪 #FactsSuperspreader,Europa,Co-founder @RENJERJerky | Forbes 30Under30 | I...,2016-02-03 13:15:55,1249.0,1472,10482,False,2021-02-10 23:54:06,This network is secured by 9 508 nodes as of t...,['BTC'],Twitter Web App,False


In [18]:
price_df.head()

,timestamp,open,high,low,close,volume,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume
0,2023-08-01 13:19:00,28902.48,28902.49,28902.48,28902.49,4.68658,1.354538e+05,258,0.89391,25836.224836
1,2023-08-01 13:18:00,28902.48,28902.49,28902.48,28902.49,4.77589,1.380351e+05,317,2.24546,64899.385195
2,2023-08-01 13:17:00,28908.52,28908.53,28902.48,28902.49,11.52263,3.330532e+05,451,2.70873,78290.170121
3,2023-08-01 13:16:00,28907.41,28912.74,28907.41,28908.53,15.89610,4.595556e+05,483,10.22981,295738.166916
4,2023-08-01 13:15:00,28896.00,28907.42,28893.03,28907.41,37.74657,1.090761e+06,686,16.50452,476955.246611


In [19]:

tweets_df['user_created'] = pd.to_datetime(tweets_df['user_created'], errors='coerce')

price_df['timestamp'] = pd.to_datetime(price_df['timestamp'], errors='coerce')

tweets_df['date'] = tweets_df['user_created'].dt.date
price_df['date'] = price_df['timestamp'].dt.date

print("\nTweet Date Conversion:")
print(tweets_df[['user_created', 'date']].head())

print("\nPrice Date Conversion:")
print(price_df[['timestamp', 'date']].head())



Tweet Date Conversion:
         user_created        date
0 2009-04-26 20:05:09  2009-04-26
1 2019-10-17 20:12:10  2019-10-17
2 2014-11-10 10:50:37  2014-11-10
3 2019-09-28 16:48:12  2019-09-28
4 2016-02-03 13:15:55  2016-02-03

Price Date Conversion:
            timestamp        date
0 2023-08-01 13:19:00  2023-08-01
1 2023-08-01 13:18:00  2023-08-01
2 2023-08-01 13:17:00  2023-08-01
3 2023-08-01 13:16:00  2023-08-01
4 2023-08-01 13:15:00  2023-08-01


In [20]:
merged_df = pd.merge(tweets_df, price_df, on='date', how='inner')


In [21]:
merged_df.head()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,...,timestamp,open,high,low,close,volume,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume
0,CryptoND,NaN,😎 BITCOINLIVE is a Dutch platform aimed at inf...,2019-10-17 20:12:10,6769.0,1532,25483,False,2019-10-17,"😎 Today, that's this #Thursday, we will do a ""...",...,2019-10-17 23:59:00,8062.82,8071.71,8061.65,8070.58,22.103764,178333.794349,198,17.359531,140058.313585
1,CryptoND,NaN,😎 BITCOINLIVE is a Dutch platform aimed at inf...,2019-10-17 20:12:10,6769.0,1532,25483,False,2019-10-17,"😎 Today, that's this #Thursday, we will do a ""...",...,2019-10-17 23:58:00,8071.84,8071.86,8061.38,8061.57,24.691655,199227.596993,241,4.360711,35183.615365
2,CryptoND,NaN,😎 BITCOINLIVE is a Dutch platform aimed at inf...,2019-10-17 20:12:10,6769.0,1532,25483,False,2019-10-17,"😎 Today, that's this #Thursday, we will do a ""...",...,2019-10-17 23:57:00,8075.01,8075.82,8071.08,8071.08,7.345408,59296.708406,191,2.453292,19805.331322
3,CryptoND,NaN,😎 BITCOINLIVE is a Dutch platform aimed at inf...,2019-10-17 20:12:10,6769.0,1532,25483,False,2019-10-17,"😎 Today, that's this #Thursday, we will do a ""...",...,2019-10-17 23:56:00,8070.88,8076.00,8070.67,8075.20,17.916791,144647.198657,228,14.015234,113153.799956
4,CryptoND,NaN,😎 BITCOINLIVE is a Dutch platform aimed at inf...,2019-10-17 20:12:10,6769.0,1532,25483,False,2019-10-17,"😎 Today, that's this #Thursday, we will do a ""...",...,2019-10-17 23:55:00,8072.63,8074.85,8070.88,8071.69,8.552252,69039.712651,191,3.160824,25517.116281


In [22]:
merged_df.shape[0]

8667996

In [23]:
def preprocess_text(text):
    text = emoji.demojize(text)
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'\d+', '', text)
    stop_words = set(stopwords.words('english'))
    text = ' '.join([word for word in word_tokenize(text) if word not in stop_words])
    lemmatizer = WordNetLemmatizer()
    text = ' '.join([lemmatizer.lemmatize(word) for word in word_tokenize(text)])
    return text


In [ ]:
merged_df['text'] = merged_df['text'].apply(preprocess_text)


In [ ]:
merged_df.head()

In [ ]:



word2vec_model = gensim.models.KeyedVectors.load_word2vec_format('/content/drive/MyDrive/NLP/word2vec_model.bin', binary=True)




with open('/content/drive/MyDrive/NLP/word2vec_logistic_model.pkl', 'rb') as file:
    sentiment_classifier = pickle.load(file)

def text_to_word2vec(text, model):
    words = text.split()
    word_vectors = [model[word] for word in words if word in model.key_to_index]

    if len(word_vectors) == 0:
        # Return a zero vector if no valid words are found
        return np.zeros(model.vector_size)
    else:
        return np.mean(word_vectors, axis=0)

# Sentiment Prediction Function
def sentiment_analysis(text):
    vector = text_to_word2vec(text, word2vec_model)
    sentiment = sentiment_classifier.predict([vector])[0]
    confidence = max(sentiment_classifier.predict_proba([vector])[0])
    return sentiment, round(confidence, 2)

# Apply Sentiment Analysis on Tweets
merged_df['sentiment_label'], merged_df['confidence_score'] = zip(
    *merged_df['text'].apply(lambda x: sentiment_analysis(x))
)

# Display sample rows with sentiment labels
print("\nDataset with Sentiment Labels:")
print(merged_df[['text', 'sentiment_label', 'confidence_score']].head())
